In [ ]:
from sklearn.linear_model import ElasticNet
import optuna.visualization as vis

import numpy as np
import optuna
import joblib
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

## Data Loading

### By Country

In [2]:
filepath = '../fs_corr_income_data'

In [3]:
test_data_x60 = pd.read_csv(filepath + '/test/60/X_test.csv')
test_data_x60 = test_data_x60.drop(columns=['Unnamed: 0'], axis=1)
test_data_y60 = pd.read_csv(filepath + '/test/60/y_test.csv')
test_data_y60 = test_data_y60.drop(columns=['Unnamed: 0'], axis=1)
test_data_60 = pd.concat([test_data_x60, test_data_y60])

test_data_x70 = pd.read_csv(filepath + '/test/70/X_test.csv')
test_data_x70 = test_data_x70.drop(columns=['Unnamed: 0'], axis=1)
test_data_y70 = pd.read_csv(filepath + '/test/70/y_test.csv')
test_data_y70 = test_data_y70.drop(columns=['Unnamed: 0'], axis=1)

test_data_x80 = pd.read_csv(filepath + '/test/80/X_test.csv')
test_data_x80 = test_data_x80.drop(columns=['Unnamed: 0'], axis=1)
test_data_y80 = pd.read_csv(filepath + '/test/80/y_test.csv')
test_data_y80 = test_data_y80.drop(columns=['Unnamed: 0'], axis=1)

In [9]:
filepath_lit = '../fs_fromlit_data'

In [ ]:
test_data_x_lit = pd.read_csv(filepath_lit + '/income/test/X_test.csv')
test_data_y_lit = pd.read_csv(filepath_lit + '/income/test/y_test.csv')
test_data_lit = pd.concat([test_data_x_lit, test_data_y_lit])

In [16]:
test_data_lit.dropna(axis=0)

,Unnamed: 0,date,setting,Women participating in own health care decisions (% of women age 15-49),Women and girls who use menstrual materials (% of women and girls ages 15-49 who had a menstrual period within the last year),"Mortality rate, infant (per 1,000 live births)",Female genital mutilation prevalence (%),Pregnant women receiving prenatal care (%),Knowledge of any method of contraception (% of all women ages 15-49),Demand for family planning satisfied by any methods (% of married women with demand for family planning),...,"School enrollment, secondary (% net) female","Literacy rate, adult total (% of people ages 15 and above) female",income_num,Borrowed for health or medical purposes (% age 15+),2.B.04 Hypertensive heart disease prevalence (age standardized) (per 100 000 population) female,2.H.0 Diabetes and kidney diseases prevalence (age standardized) (per 100 000 population) female,Probability of Survival to Age 5,Population living in areas where elevation is below 5 meters (% of total population) rural,Coverage of social safety net programs (% of population) Quintile 1 (poorest),"Maternal mortality ratio (national estimate, per 100,000 live births)"


In [21]:
test_data_x60.dropna(axis=1)

,"Mortality rate, infant (per 1,000 live births)","Survival to age 65, female (% of cohort)","Mortality rate, under-5, male (per 1,000 live births)","Mortality rate, infant, female (per 1,000 live births)","Mortality rate, under-5, female (per 1,000 live births)","Mortality rate, infant, male (per 1,000 live births)","Mortality rate, adult, female (per 1,000 female adults)","Fertility rate, total (births per woman)","Mortality rate, under-5 (per 1,000 live births)"
0,66.9,64.624,110.0,59.9,98.0,73.4,192.423,5.992,104.2
1,3.0,92.576,4.0,2.7,3.3,3.2,47.006,1.280,3.7
2,17.4,83.728,21.7,15.3,17.2,19.4,96.702,1.825,19.5
3,6.7,90.158,8.5,6.1,7.1,7.3,59.505,1.676,7.8
4,30.5,77.321,40.4,27.8,34.1,33.0,150.212,2.721,37.3
...,...,...,...,...,...,...,...,...,...
329,13.1,84.176,15.6,12.8,14.7,13.3,93.526,2.936,15.2
330,15.4,84.745,19.1,13.6,15.2,17.1,91.547,1.767,17.2
331,22.2,79.406,28.8,20.0,23.7,24.3,124.175,2.546,26.3
332,5.4,84.565,7.2,4.8,5.8,6.0,93.451,1.800,6.5


In [22]:
validation_60 = {}

for fold in range(0, 5):
    vdata_x60 = pd.read_csv(filepath + '/val/60/X_val_' + str(fold) + '.csv')
    vdata_x60 = vdata_x60.drop(columns=['Unnamed: 0'], axis=1)
    vdata_y60 = pd.read_csv(filepath + '/val/60/y_val_' + str(fold) + '.csv')
    vdata_y60 = vdata_y60.drop(columns=['Unnamed: 0'], axis=1)
    validation_60[fold] = [vdata_x60, vdata_y60]

In [39]:
validation_60[2][0].dropna(axis=1)

,"Survival to age 65, female (% of cohort)","Mortality rate, adult, female (per 1,000 female adults)","Fertility rate, total (births per woman)"
0,79.970,104.041,1.820
1,88.436,72.268,2.988
2,82.535,101.816,2.171
3,82.578,99.275,2.148
4,83.297,94.902,1.960
...,...,...,...
486,74.866,153.848,1.539
487,78.993,135.710,1.692
488,89.154,72.654,1.712
489,74.237,153.690,2.388


In [23]:
train_60 = {}

for fold in range(0, 5):
    tdata_x1 = pd.read_csv(filepath + '/train/60/X_train_' + str(fold) + '_1.csv')
    tdata_x1 = tdata_x1.drop(columns=['Unnamed: 0'], axis=1)
    tdata_y1 = pd.read_csv(filepath + '/train/60/y_train_' + str(fold) + '_1.csv')
    tdata_y1 = tdata_y1.drop(columns=['Unnamed: 0'], axis=1)

    train_60[fold] = [tdata_x1, tdata_y1]

### By Year

In [24]:
filepath_year = '../fs_corr_year_data'

In [25]:
test_data_x60_year = pd.read_csv(filepath_year + '/test/60/X_test.csv')
test_data_x60_year = test_data_x60_year.drop(columns=['Unnamed: 0'], axis=1)
test_data_y60_year = pd.read_csv(filepath_year + '/test/60/y_test.csv')
test_data_y60_year = test_data_y60_year.drop(columns=['Unnamed: 0'], axis=1)
test_data_60_year = pd.concat([test_data_x60_year, test_data_y60_year])

test_data_x70_year = pd.read_csv(filepath_year + '/test/70/X_test.csv')
test_data_x70_year = test_data_x70_year.drop(columns=['Unnamed: 0'], axis=1)
test_data_y70_year = pd.read_csv(filepath_year + '/test/70/y_test.csv')
test_data_y70_year = test_data_y70_year.drop(columns=['Unnamed: 0'], axis=1)

test_data_x80_year = pd.read_csv(filepath_year + '/test/80/X_test.csv')
test_data_x80_year = test_data_x80_year.drop(columns=['Unnamed: 0'], axis=1)
test_data_y80_year = pd.read_csv(filepath_year + '/test/80/y_test.csv')
test_data_y80_year = test_data_y80_year.drop(columns=['Unnamed: 0'], axis=1)

In [31]:
test_data_x60_year.dropna(axis=1)

,"Survival to age 65, female (% of cohort)","Fertility rate, total (births per woman)"
0,64.547,4.309
1,69.528,4.469
2,95.498,1.172
3,86.729,2.133
4,87.522,1.863
...,...,...
198,87.810,2.053
199,72.944,2.766
200,82.598,2.126
201,77.736,2.025


In [32]:
filepath_lit_year = '../fs_fromlit_data'
test_data_x_lit_year = pd.read_csv(filepath_lit_year + '/year/test/X_test.csv')
test_data_y_lit_year = pd.read_csv(filepath_lit_year + '/year/test/y_test.csv')
test_data_lit_year = pd.concat([test_data_x_lit_year, test_data_y_lit_year])

In [34]:
test_data_lit_year.dropna(axis=0)

,Unnamed: 0,date,setting,Women participating in own health care decisions (% of women age 15-49),Women and girls who use menstrual materials (% of women and girls ages 15-49 who had a menstrual period within the last year),"Mortality rate, infant (per 1,000 live births)",Female genital mutilation prevalence (%),Pregnant women receiving prenatal care (%),Knowledge of any method of contraception (% of all women ages 15-49),Demand for family planning satisfied by any methods (% of married women with demand for family planning),...,"School enrollment, secondary (% net) female","Literacy rate, adult total (% of people ages 15 and above) female",income_num,Borrowed for health or medical purposes (% age 15+),2.B.04 Hypertensive heart disease prevalence (age standardized) (per 100 000 population) female,2.H.0 Diabetes and kidney diseases prevalence (age standardized) (per 100 000 population) female,Probability of Survival to Age 5,Population living in areas where elevation is below 5 meters (% of total population) rural,Coverage of social safety net programs (% of population) Quintile 1 (poorest),"Maternal mortality ratio (national estimate, per 100,000 live births)"


## Linear Regression

In [35]:
output_dir_lin_inc = '../regression/linear/country'

In [ ]:
def objective_linreg(trial):

    alpha = trial.suggest_float("alpha", 0.1, 1.0)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)
    
    #to get the predictions of the base models 
    #fit_intercept=True
    elastic_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True, random_state=42)
    elastic_model.fit(X_meta_train, y_meta_train)
    elastic_predictions = elastic_model.predict(X_meta_val)

    return mean_squared_error(y_meta_val, elastic_predictions)

In [ ]:
for fold in range(0, 5):
    val_input_data = validation[fold][0].copy()
    val_input_data['setting'] = val_input_data['setting'].astype("category")
    val_input_data.columns = val_input_data.columns.str.replace(r'[\"\[\]\{\}\\:,]', '', regex=True)
    val_label = validation[fold][1].copy()
    val_label.column = 'Maternal mortality ratio (national estimate per 100000 live births)'

    train_input_data = train[fold][thresh_idx * 2].copy()
    train_input_data['setting'] = train_input_data['setting'].astype("category")
    train_input_data.columns = train_input_data.columns.str.replace(r'[\"\[\]\{\}\\:,]', '', regex=True)
    train_label = train[fold][thresh_idx * 2 + 1].copy()
    train_label.column = 'Maternal mortality ratio (national estimate per 100000 live births)'

    #Create a study object and optimize the objective function.
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, train_input_data, train_label, val_input_data, val_label), n_trials=300)
    best_model = lgb.LGBMRegressor(**study.best_params)
    best_model.fit(train_input_data, train_label)
    
    #save best model 
    joblib.dump(best_model, output_dir + '/best_model_' + str(fold) + '_' + thresh +  '.pkl')
    joblib.dump(study.best_params, f"{output_dir}/best_params_{fold}_{thresh}.pkl")

    # Save study for later visualization
    joblib.dump(study, f"{output_dir}/optuna_study_{fold}_{thresh}.pkl")

    summary = {
        "dataset": str(fold) + '_' + thresh,
        "fold" : fold,
        "threshold": thresh,
        "model": 'lightgbm',
        "best_params": study.best_params,
        "best_optuna_loss": study.best_value
    }

    with open(f"{output_dir}/results_{fold}_{thresh}.json", "w") as f:
        json.dump(summary, f, indent=2)